In [3]:
import cv2
import numpy as np

In [4]:
class PointCloud:
    def __init__(self):
        # Dummy initialization
        self.points = np.zeros((100, 100, 3), dtype=np.float32)
        self.colors = np.zeros((100, 100, 3), dtype=np.uint8)
        self.normals = np.zeros((100, 100, 3), dtype=np.float32)

    def clear(self):
        self.points = np.zeros_like(self.points)
        self.colors = np.zeros_like(self.colors)
        self.normals = np.zeros_like(self.normals)

    def init_points(self, rows, cols):
        self.points = np.full((rows, cols, 3), np.nan, dtype=np.float32)

    def init_color(self, rows, cols):
        self.colors = np.full((rows, cols, 3), 255, dtype=np.uint8)

    def init_normals(self, rows, cols):
        self.normals = np.full((rows, cols, 3), np.nan, dtype=np.float32)


In [5]:
def triangulate_stereo(K1, kc1, K2, kc2, Rt, T, p1, p2):
    # To image camera coordinates
    inp1 = np.array([p1.x, p1.y], dtype=np.float64).reshape((1, 1, 2))
    inp2 = np.array([p2.x, p2.y], dtype=np.float64).reshape((1, 1, 2))

    outp1 = cv2.undistortPoints(inp1, K1, kc1)
    outp2 = cv2.undistortPoints(inp2, K2, kc2)

    assert outp1.dtype == np.float64 and outp1.shape == (1, 1, 2)
    assert outp2.dtype == np.float64 and outp2.shape == (1, 1, 2)

    outvec1 = outp1[0, 0]
    outvec2 = outp2[0, 0]

    u1 = np.array([outvec1[0], outvec1[1], 1.0], dtype=np.float64)
    u2 = np.array([outvec2[0], outvec2[1], 1.0], dtype=np.float64)

    # To world coordinates
    w1 = u1
    w2 = np.dot(Rt, (u2 - T).reshape(-1, 1)).reshape(-1)

    # World rays
    v1 = w1
    v2 = np.dot(Rt, u2).reshape(-1)

    # Compute ray-ray approximate intersection
    p3d, distance, _, _ = approximate_ray_intersection(v1, w1, v2, w2)

    return p3d, distance




In [6]:
def reconstruct_model_patch_center(pointcloud, calib, pattern_image, min_max_image, color_image, projector_size, threshold, max_dist):
    if pattern_image is None or pattern_image.dtype != np.float32 or pattern_image.shape[2] != 2:
        print("[reconstruct_model] ERROR invalid pattern_image")
        return

    if min_max_image is None or min_max_image.dtype != np.uint8 or min_max_image.shape[2] != 2:
        print("[reconstruct_model] ERROR invalid min_max_image")
        return

    if color_image is not None and (color_image.dtype != np.uint8 or color_image.shape[2] != 3):
        print("[reconstruct_model] ERROR invalid color_image")
        return

    if not calib.is_valid():
        # invalid calibration
        return

    plane_dist = 100.0

    # Initialize point cloud
    scale_factor_x = 1
    scale_factor_y = 1 if projector_size[0] > projector_size[1] else 2
    out_cols = projector_size[0] // scale_factor_x
    out_rows = projector_size[1] // scale_factor_y

    pointcloud.clear()
    pointcloud.init_points(out_rows, out_cols)
    pointcloud.init_color(out_rows, out_cols)

    # Progress
    progress = None

    # Candidate points
    proj_points = {}
    cam_points = {}

    # Iterate through pattern image
    for h in range(pattern_image.shape[0]):
        if progress and h % 4 == 0:
            progress.setValue(h)
            progress.setLabelText("Reconstruction in progress: collecting points")
        
        if progress and progress.wasCanceled():
            # Abort
            pointcloud.clear()
            return

        curr_pattern_row = pattern_image[h]
        min_max_row = min_max_image[h]
        
        for w in range(pattern_image.shape[1]):
            pattern = curr_pattern_row[w]
            min_max = min_max_row[w]

            if np.any(np.isnan(pattern)) or pattern[0] < 0 or pattern[0] >= projector_size[0] or pattern[1] < 0 or pattern[1] >= projector_size[1] or (min_max[1] - min_max[0]) < threshold:
                # Skip invalid points
                continue

            # Add valid points
            proj_point = np.array([pattern[0] / scale_factor_x, pattern[1] / scale_factor_y])
            index = int(proj_point[1]) * out_cols + int(proj_point[0])

            if index not in proj_points:
                proj_points[index] = proj_point
                cam_points[index] = []

            cam_points[index].append(np.array([w, h]))

    if progress:
        progress.setValue(pattern_image.shape[0])

    Rt = np.transpose(calib.R)

    if progress:
        progress.setMaximum(len(proj_points))

    # Iterate through projected points
    for index, proj_point in proj_points.items():
        if progress and index % 1000 == 0:
            progress.setValue(index)
            progress.setLabelText("Reconstruction in progress: {} good points/{} bad points".format(good, bad))
        
        if progress and progress.wasCanceled():
            # Abort
            pointcloud.clear()
            return

        cam_point_list = cam_points[index]
        count = len(cam_point_list)

        if not count:
            # Empty list
            continue

        # Center average
        sum_x, sum_y = np.sum(cam_point_list, axis=0)
        sum2_x, sum2_y = np.sum(cam_point_list**2, axis=0)
        cam = np.array([sum_x/count, sum_y/count])
        proj = np.array([proj_point[0] * scale_factor_x, proj_point[1] * scale_factor_y])

        # Triangulate
        distance = max_dist
        p = np.zeros(3)
        triangulate_stereo(calib.cam_K, calib.cam_kc, calib.proj_K, calib.proj_kc, Rt, calib.T, cam, proj, p, distance)

        if distance < max_dist:
            # Good point
            d = plane_dist + 1

            # Evaluate the plane
            if d > plane_dist:
                # Object point, keep
                good += 1
                cloud_point = pointcloud.points[proj_point[1], proj_point[0]]
                cloud_point[0] = p[0]
                cloud_point[1] = p[1]
                cloud_point[2] = p[2]

                if color_image is not None:
                    vec = color_image[int(cam[1]), int(cam[0])]
                    cloud_color = pointcloud.colors[proj_point[1], proj_point[0]]
                    cloud_color[0] = vec[0]
                    cloud_color[1] = vec[1]
                    cloud_color[2] = vec[2]
        else:
            # Skip
            bad += 1

    if progress:
        progress.setValue(len(proj_points))
        progress.close()

    print("Reconstructed points [patch center]: {} ({} skipped, {} invalid)".format(good, bad, invalid))

# Dummy classes to represent the missing parts
class CalibrationData:
    def __init__(self):
        # Dummy initialization
        self.R = np.eye(3)
        self.T = np.zeros((3, 1))
        self.cam_K = np.eye(3)
        self.cam_kc = np.zeros(5)
        self.proj_K = np.eye(3)
        self.proj_kc = np.zeros(5)

    def is_valid(self):
        # Dummy implementation
        return True

class PointCloud:
    def __init__(self):
        # Dummy initialization
        self.points = np.zeros((100, 100, 3))
        self.colors = np.zeros((100, 100, 3))

    def clear(self):
        # Dummy implementation
        pass

    def init_points(self, rows, cols):
        # Dummy implementation
        pass

    def init_color(self, rows, cols):
        # Dummy implementation
        pass


# Usage
pointcloud = PointCloud()
calib = CalibrationData()
pattern_image = np.random.rand(100, 100, 2).astype(np.float32)
min_max_image = np.random.randint(0, 255, size=(100, 100, 2), dtype=np.uint8)



In [8]:
class ProjectorSize:
    def __init__(self):
        self.width = 1920
        self.height= 1080


        def __del__(self):
            # Destructor
            pass
pointC = PointCloud()

projectorSize = ProjectorSize()

threshold=50


In [17]:
class Scan3D:
    class CalibrationData:
        def __init__(self):
            self.cam_K = np.eye(3)
            self.cam_kc = np.zeros(5)
            self.proj_K = np.eye(3)
            self.proj_kc = np.zeros(5)
            self.R = np.eye(3)
            self.T = np.zeros((3, 1))
            self.cam_error = 0.0
            self.proj_error = 0.0
            self.stereo_error = 0.0
            self.filename = ""

        def set_calibration_data(self, R, T, proj_matrix, cam_matrix):
            self.R = R
            self.T = T
            self.proj_K = proj_matrix[:, :3]
            self.proj_kc = proj_matrix[:, 3:]
            self.cam_K = cam_matrix[:, :3]
            self.cam_kc = cam_matrix[:, 3:]

# Load calibration data
coeiff = np.load('calib.npz')
R = coeiff['R']
T = coeiff['T']
proj_matrix = coeiff['proj_matrix']
cam_matrix = coeiff['cam_matrix']

# Create Scan3D object and set calibration data
scan3d_instance = Scan3D()
scan3d_instance.CalibrationData.set_calibration_data(scan3d_instance, R, T, proj_matrix, cam_matrix)


minMaxImage = None
colorImage = None
maxDist = 10

In [18]:
reconstruct_model_patch_center(pointcloud, scan3d_instance.CalibrationData, pattern_image, minMaxImage, colorImage, projectorSize, threshold, maxDist)

[reconstruct_model] ERROR invalid min_max_image
